<a href="https://colab.research.google.com/github/parveenbanurajak/Java_basic_code/blob/main/Traffic_Prediction_Prophet%2C_Sarima_and_Sarimax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
fedesoriano_traffic_prediction_dataset_path = kagglehub.dataset_download('fedesoriano/traffic-prediction-dataset')

print('Data source import complete.')


## <center> Road Traffic Prediction : EDA, Prophet, SARIMA and SARIMAX </center>

#### Context
Traffic congestion is increasing in cities worldwide, driven by growing urban populations, outdated infrastructure, poorly coordinated traffic signals, and a lack of real-time data. The consequences are substantial. According to traffic data and analytics firm INRIX, U.S. commuters lost $305 billion in 2017 due to wasted fuel, lost time, and higher costs of transporting goods through congested areas. With limited physical space and financial resources to build more roads, cities need to adopt new strategies and technologies to enhance traffic management.
#### Content
This dataset contains 48.1k (48120) observations of the number of vehicles each hour in four different junctions. In this project, we'll focus solely on data from Junction 1.

In [ ]:
# !pip install prophet
# !pip install holidays

In [ ]:
pip install pmdarima

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.api.types import CategoricalDtype
import holidays
from sklearn.linear_model import LinearRegression
from datetime import datetime
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings("ignore")

## Import time series data

In [ ]:
df = pd.read_csv('/kaggle/input/traffic-prediction-dataset/traffic.csv',index_col=[0],parse_dates=[0])
df.head()

In [ ]:
df.shape

## Pre-processing

In [ ]:
df = df[df['Junction'] == 1]             # Filter for Junction 1
df = df.drop(columns=['ID', 'Junction']) # Drop the 'ID' and 'Junction' columns
df.head()

In [ ]:
df.shape

In [ ]:
# check for missing values and fill them if there
if(df.isnull().sum().sum()==0):
    print('No missing values.')
else:
    df.fillna(method='ffill',inplace=True)

## Feature engineering

In [ ]:
cat_type = CategoricalDtype(categories=['Monday','Tuesday','Wednesday',
                                        'Thursday','Friday','Saturday','Sunday'],
                            ordered=True)
us_holidays = holidays.US(years=[2015, 2016, 2017])

def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['weekday'] = df['date'].dt.day_name()
    df['weekday'] = df['weekday'].astype(cat_type)
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['isholiday'] = df['date'].dt.normalize().isin(us_holidays)
    df['date_offset'] = (df.date.dt.month*100 + df.date.dt.day - 320)%1300

    df['season'] = pd.cut(df['date_offset'], [0, 300, 602, 900, 1300],
                          labels=['Spring', 'Summer', 'Fall', 'Winter']
                   )
    X = df[['hour','isholiday','weekday','month','year','season']]
    if label:
        y = df[label]
        return X, y
    return X

X, y = create_features(df, label='Vehicles')
features_and_target = pd.concat([X, y], axis=1)

# Display the first few rows to verify
print(features_and_target[features_and_target['isholiday'] == True])

## Data Visualization

In [ ]:
# For better readability, resample the data by day and take the mean
plt.figure(figsize=(10, 5))
df_daily = df.resample('D').mean()
plt.plot(df_daily.index, df_daily['Vehicles'])
plt.title('Traffic Volume Over Time')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.ylabel('Average Number of Vehicles')
plt.show()

In [ ]:
# Years of traffic
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
year_counts = features_and_target['year'].value_counts().sort_index()
plt.figure(figsize=(10, 5))
plt.bar(year_counts.index.astype(str), year_counts.values, color=colors[:len(year_counts)])
plt.title("Years of Traffic")
plt.xlabel("Year")
plt.ylabel("Number of Records")
plt.show()

In [ ]:
# Peak Hours Identification
plt.figure(figsize=(10, 5))
avg_traffic_by_hour = features_and_target.groupby('hour')['Vehicles'].mean()
avg_traffic_by_hour.plot(kind='bar')
plt.title('Average Traffic Volume by Hour of Day')
plt.xlabel('Hour of Day')
plt.ylabel('Average Traffic Volume')
plt.show()

In [ ]:
# Determine how traffic volumes vary by day of the week
plt.figure(figsize=(10, 5))
average_traffic_by_day = features_and_target.groupby('weekday')['Vehicles'].mean()
average_traffic_by_day.plot(kind='bar', color='lightgreen')
plt.title('Average Traffic Volume by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Average Traffic Volume')
plt.show()

In [ ]:
# Determine how traffic volumes differ vary by Day of Week and Season
import seaborn as sns
fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(data=features_and_target.dropna(),
            x='weekday',
            y='Vehicles',
            hue='season',
            ax=ax,
            linewidth=1)
ax.set_title('Vehicles by Day of Week')
ax.set_xlabel('Day of Week - Junction 1')
ax.set_ylabel('Vehicles')
ax.legend(bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
# Assess how public holidays and other significant days affect traffic volumes
average_volume = features_and_target.groupby('isholiday')['Vehicles'].mean()
plt.figure(figsize=(10, 6))
average_volume.plot(kind='bar')
plt.title('Average Traffic Volume: Holidays vs. Non-Holidays')
plt.xlabel('Is Holiday')
plt.ylabel('Average Number of Vehicles')
plt.xticks(ticks=[0, 1], labels=['Non-Holiday', 'Holiday'], rotation=0)
plt.show()

## Time Series Decomposition

In [ ]:
# Perform the seasonal decomposition
result = seasonal_decompose(df_daily['Vehicles'], model='additive', period=7)
result.plot()
plt.tight_layout()
plt.show()

- The traffic volume data shows a repeating pattern, indicating seasonality, likely weekly.
- **The trend component** suggests periods of increase and decrease in traffic, though specific conclusions require more context.
- **The seasonal component** reveals consistent peaks and troughs, likely tied to workweek patterns.
- **Residuals**, representing irregularities, are mostly random with a few outliers, indicating that the model effectively captures trend and seasonality.

Overall, the analysis highlights a strong seasonal pattern, useful for both long-term planning and short-term traffic management.

## Long-term Trends
Analyze if there is a long-term increase or decrease in traffic volumes, which could be indicative of urban development or changes in population density

In [ ]:
# Resample the data to get monthly averages
df_monthly = df.resample('M').mean()

# Linear Regression for Trend Analysis
X = np.arange(len(df_monthly)).reshape(-1, 1)  # Time as an independent variable
y = df_monthly['Vehicles'].values  # Traffic volume as the dependent variable

# Create and fit the model
model = LinearRegression()
model.fit(X, y)

# Predict the trend
trend = model.predict(X)

# Plot the trend
plt.figure(figsize=(10, 5))
plt.plot(df_monthly.index, y, label='Actual')
plt.plot(df_monthly.index, trend, label='Trend', color='red')
plt.title('Trend Line for Monthly Average Traffic Volume')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.ylabel('Average Number of Vehicles')
plt.legend()
plt.show()

The linear trend line highlights a **long-term upward trend** in traffic volumes, likely driven by urban development, population growth, and changing traffic patterns, underscoring the need for improved infrastructure and transportation planning.

This rising trend highlights the importance of adapting traffic management strategies and provides valuable insights for city planners and policymakers to consider infrastructure improvements and public transportation enhancements to alleviate potential congestion.

## Impact of External Factors
Detect and visualize significant deviations (anomalies) in daily traffic volume - like nearby construction projects or new traffic regulations - by calculating the Z-score and setting a threshold.

In [ ]:
# Calculate the Z-score for each day's traffic volume
df_daily['z_score'] = (df_daily['Vehicles'] - df_daily['Vehicles'].mean()) / df_daily['Vehicles'].std()

# Identify days where the Z-score is above a certain threshold (e.g., 2 or -2 for significant anomalies)
anomalies = df_daily[abs(df_daily['z_score']) > 2]

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(df_daily.index, df_daily['Vehicles'], label='Daily Traffic Volume')
plt.scatter(anomalies.index, anomalies['Vehicles'], color='red', label='Anomalies')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.ylabel('Average Daily Traffic Volume')
plt.title('Traffic Volume Over Time with Anomalies')
plt.legend()
plt.show()

- Anomalies : The red dots represent days that have been identified as anomalies, where the traffic volume was significantly different from the norm. These could be days with exceptionally high traffic volume compared to the average on other day
- Potential External Factors : The anomalies could be due to external factors such as road closures, construction projects, accidents, events, or changes in traffic regulations. These would need to be investigated by cross-referencing the dates of the anomalies with external event data.

## Train / Test Split

In [ ]:
df_daily = df.resample('D').mean()
split_date = '1-Jan-2017'
df_train = df_daily.loc[df_daily.index <= split_date].copy()
df_test = df_daily.loc[df_daily.index > split_date].copy()

In [ ]:
# Plot train and test so we can see where we have split
df_test \
    .rename(columns={'Vehicles': 'TEST SET'}) \
    .join(df_train.rename(columns={'Vehicles': 'TRAINING SET'}),
          how='outer') \
    .plot(figsize=(10, 5), title='Vehicles - Junction 1', style='-', ms=1)
plt.xlabel('Date')
plt.ylabel('Vehicles')
plt.show()

## Prophet Model

Prophet model expects the dataset to be named a specific way. We will rename our dataframe columns before feeding it into the model.
- Datetime column named: ds
- target : y

In [ ]:
df_train.head()

In [ ]:
# Format data for prophet model using ds and y
# Resetting the index converts the datetime index into a regular column
df_train_prophet = df_train.reset_index().rename(columns={'DateTime': 'ds', 'Vehicles': 'y'})
df_train_prophet.head()

#### Fit

In [ ]:
%%time
model = Prophet()
model.fit(df_train_prophet)

#### Predictions

In [ ]:
df_test_prophet = df_test.reset_index() \
    .rename(columns={'DateTime':'ds',
                     'Vehicles':'y'})
df_test_fcst = model.predict(df_test_prophet)
df_test_fcst.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(df_train_prophet['ds'], df_train_prophet['y'], label='Train Data', color='green', linewidth=2)
ax.plot(df_test_fcst['ds'], df_test_fcst['yhat'], label='Forecast', color='blue', linewidth=2)

ax.set_title('#Vehicles - Prophet Forecast')
plt.xlabel('Date')
plt.ylabel('Vehicles')
ax.legend(loc='upper left')
plt.show()

In [ ]:
fig = model.plot_components(df_test_fcst)
plt.show()

#### Compare Forecast to Actuals

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
ax.plot(df_train.index, df_train['Vehicles'], color='orange', label='Train Data', linestyle='-')
ax.plot(df_test.index, df_test['Vehicles'], color='red', label='Test Data', linestyle='-')
ax.plot(df_test_fcst['ds'], df_test_fcst['yhat'], color='blue', label='Forecast', linestyle='-')
ax.legend()
ax.set_title('# Vehicles Junction1 - Prophet Forecast')
ax.set_xlabel('Date')
ax.set_ylabel('Vehicles')
plt.show()

In [ ]:
start_date = datetime.strptime('01-01-2017', '%m-%d-%Y')
end_date = datetime.strptime('02-01-2017', '%m-%d-%Y')

plt.figure(figsize=(10, 5))

# Plotting the actual values
plt.plot(df_test.index, df_test['Vehicles'], 'r-', label="Actual")  # 'r-' for red line plot

# Plotting the forecast
model.plot(df_test_fcst, ax=plt.gca())  # Ensure forecast is on the same plot

# Setting the x-axis bounds
plt.xlim([start_date, end_date])

# Filtering legend entries to remove 'Observed data points'
handles, labels = plt.gca().get_legend_handles_labels()
filtered_handles_labels = [(h, l) for h, l in zip(handles, labels) if l != 'Observed data points']
filtered_handles, filtered_labels = zip(*filtered_handles_labels)
plt.legend(filtered_handles, filtered_labels, loc='upper left')

# Setting the labels and title
plt.xlabel('Date')
plt.ylabel('Vehicles')
plt.suptitle('January 2017 - Forecast vs Actuals')

# Display the plot
plt.show()

In [ ]:
start_date = datetime.strptime('01-02-2017', '%m-%d-%Y')
end_date = datetime.strptime('01-08-2017', '%m-%d-%Y')

plt.figure(figsize=(10, 5))

# Plotting actual values
plt.plot(df_test.index, df_test['Vehicles'], 'r-', label="Actual")  # 'r-' means red line

# Plotting the forecast
model.plot(df_test_fcst, ax=plt.gca())  # Plotting forecast using the model

# Setting the bounds for the x-axis
plt.xlim([start_date, end_date])

# Adding title and labels
plt.title('First Week of January 2017 - Forecast vs Actuals')
plt.xlabel('Date')
plt.ylabel('Vehicles')

# Handling the legend
handles, labels = plt.gca().get_legend_handles_labels()
filtered_handles_labels = [(h, l) for h, l in zip(handles, labels) if l != 'Observed data points']
filtered_handles, filtered_labels = zip(*filtered_handles_labels)
plt.legend(filtered_handles, filtered_labels, loc='upper left')

# Display the plot
plt.show()

#### Evaluate the model

In [ ]:
mae = mean_absolute_error(y_true=df_test['Vehicles'],
                   y_pred=df_test_fcst['yhat'])
mape = mean_absolute_percentage_error(y_true=df_test['Vehicles'],
                   y_pred=df_test_fcst['yhat'])*100
rmse = np.sqrt(mean_squared_error(y_true=df_test['Vehicles'],
                   y_pred=df_test_fcst['yhat']))

performance = pd.DataFrame({
    'Metrics': ['MAE', 'MAPE', 'RMSE'],
    'Prophet': [mae, f'{mape:.2f}%', rmse],
})

print(performance.set_index('Metrics'))

#### Adding Holidays

We will see if adding holiday indicators will help the accuracy of the model. Prophet comes with a Holiday Effects parameter that can be provided to the model prior to training.

In [ ]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

cal = calendar()
holidays = cal.holidays(start=df_daily.index.min(),
                        end=df_daily.index.max(),
                        return_name=True)
holiday_df = pd.DataFrame(data=holidays,
                          columns=['holiday'])
holiday_df = holiday_df.reset_index().rename(columns={'index':'ds'})

In [ ]:
holiday_df.head()

In [ ]:
%%time
model_with_holidays = Prophet(holidays=holiday_df)
model_with_holidays.fit(df_train_prophet)

In [ ]:
df_test_fcst_with_hols = model_with_holidays.predict(df=df_test_prophet)

In [ ]:
mae_with_hols = mean_absolute_error(y_true=df_test,
                   y_pred=df_test_fcst_with_hols['yhat'])
mape_with_hols = mean_absolute_percentage_error(y_true=df_test,
                   y_pred=df_test_fcst_with_hols['yhat'])*100
rmse_with_hols = np.sqrt(mean_squared_error(y_true=df_test,
                   y_pred=df_test_fcst_with_hols['yhat']))

performance = pd.DataFrame({
    'Metrics': ['MAE', 'MAPE', 'RMSE'],
    'Prophet': [mae, f'{mape:.2f}%', rmse],
    'Prophet(holidays)': [mae_with_hols, f'{mape_with_hols:.2f}%', rmse_with_hols]
})

print(performance.set_index('Metrics'))

The model with holidays is slightly better at capturing patterns in the data.

#### Predict into the Future

In [ ]:
%%time
df_prophet = df_daily.reset_index() \
    .rename(columns={'DateTime':'ds',
                     'Vehicles':'y'})
model_with_holidays = Prophet(holidays=holiday_df)
model_fulldf = model_with_holidays.fit(df_prophet)

In [ ]:
# Generates future dates starting from the last date in df.
future_data = model_fulldf.make_future_dataframe(
                    periods=365*1,
#                     freq='h',
                    include_history=False)
next_forecast = model_fulldf.predict(future_data)
next_forecast[['ds','yhat']].head()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df_daily.index, df_daily['Vehicles'], label='Historical Data', color='blue')
plt.plot(next_forecast['ds'], next_forecast['yhat'], label='Forecast', color='green')
plt.xlabel('Date')
plt.ylabel('Vehicles')
plt.title('Forecast')
plt.legend()
plt.show()

## SARIMA Model

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.api as smt
from tabulate import tabulate
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from itertools import product
from tqdm.notebook import tqdm

#### Check stationary

In [ ]:
df_diff = df_daily.diff().dropna()
result = adfuller(df_diff)
table = [
    ['ADF Statistic', result[0]],
    ['P-value', result[1]],
    ['Conclusion', 'The time series is stationary' if result[1] < 0.05 else 'The time series is not stationary']
]
print(tabulate(table, headers=['Metrics', 'Value'], tablefmt='github'))

#### Define orders

In [ ]:
max_lags = 30  # Maximum allowable lags

acf_vals = acf(df_diff, nlags=max_lags)
plt.bar(range(len(acf_vals)), acf_vals, color='blue')
plt.title('Autocorrelation Function (ACF)')
plt.xlabel('Lag')
plt.ylabel('ACF Value')
plt.show()

pacf_vals = pacf(df_diff, nlags=max_lags)
plt.bar(range(len(pacf_vals)), pacf_vals, color='green')
plt.title('Partial Autocorrelation Function (PACF)')
plt.xlabel('Lag')
plt.ylabel('PACF Value')
plt.show()

In [ ]:
# setting initial values and some bounds for them
ps = range(0, 3)
d=1
qs = range(0, 3)
Ps = range(0, 3)
D=1
Qs = range(0, 3)
s = 7 # season length

# creating list with all the possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
def optimizeSARIMA(parameters_list, d, D, s, df):
    """
        Return dataframe with parameters and corresponding AIC

        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order
        s - length of season
        df - the DataFrame containing the time series data
    """

    results = []
    best_aic = float("inf")

    for param in tqdm(parameters_list):
        # we need try-except because on some combinations model fails to converge
        try:
            model=sm.tsa.statespace.SARIMAX(df, order=(param[0], d, param[1]),
                                            seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    # sorting in ascending order, the lower AIC is - the better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)

    return result_table

In [ ]:
%%time
result_table = optimizeSARIMA(parameters_list, d, D, s, df_train)

In [ ]:
result_table.head()

#### Fit the model

In [ ]:
# set the parameters that give the lowest AIC
p, q, P, Q = result_table.parameters[0]

best_model=sm.tsa.statespace.SARIMAX(df_train, order=(p, d, q),
                                        seasonal_order=(P, D, Q, s)).fit(disp=-1)
print(best_model.summary())

#### Residual Analysis

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    """
        Plot time series, its ACF and PACF, calculate Dickey–Fuller test

        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    with plt.style.context(style):
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))

        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
tsplot(best_model.resid[7+1:], lags=30)

#### Predictions

In [ ]:
forecast = best_model.get_prediction(start=df_test.index[0], end=df_test.index[-1], dynamic=False)
forecast_mean = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()

In [ ]:
plt.figure(figsize=(10, 5))

# Plotting the training data
plt.plot(df_train.index, df_train, label='Train Data', color='blue')

# Plotting the actual test data
plt.plot(df_test.index, df_test, label='Test Data', color='green')

# Plotting the forecasted data
plt.plot(forecast_mean.index, forecast_mean, label='Forecast', color='red')

# Plotting the confidence intervals
plt.fill_between(forecast_conf_int.index,
                 forecast_conf_int.iloc[:, 0],
                 forecast_conf_int.iloc[:, 1], color='pink', alpha=0.3, label='Uncertainty Interval')

# Final plot adjustments
plt.title('SARIMA Model: Train, Test, and Forecast')
plt.xlabel('Date')
plt.ylabel('Vehicles')
plt.legend()
plt.show()

#### Compare Prophet and SARIMA

In [ ]:
mae_sarima = mean_absolute_error(df_test, forecast_mean)
mape_sarima = mean_absolute_percentage_error(df_test, forecast_mean) * 100
rmse_sarima = np.sqrt(mean_squared_error(df_test, forecast_mean))

performance = pd.DataFrame({
    'Metrics': ['MAE', 'MAPE', 'RMSE'],
    'Prophet(holidays)': [mae_with_hols, f'{mape_with_hols:.2f}%', rmse_with_hols],
    'SARIMA': [mae_sarima, f'{mape_sarima:.2f}%', rmse_sarima]
})

print(performance.set_index('Metrics'))

Prophet outperforms SARIMA in forecasting traffic data due to its superior ability to capture trends, seasonality, and holiday effects, leading to lower error metrics. This suggests Prophet is better suited for the complexities of this dataset.

## SARIMAX Model

In [ ]:
from holidays import US
us_holidays = US(years=[2015, 2016, 2017])

# Create copies of the original DataFrames before adding the 'isholiday' column
df_train_holidays = df_train.copy()
df_test_holidays = df_test.copy()

# Add the 'isholiday' column to the copied DataFrames
df_train_holidays['isholiday'] = df_train_holidays.index.normalize().isin(us_holidays).astype(int)
df_test_holidays['isholiday'] = df_test_holidays.index.normalize().isin(us_holidays).astype(int)

# Create exogenous matrices for holidays
exog_train = df_train_holidays[['isholiday']]
exog_test = df_test_holidays[['isholiday']]

# Fit the SARIMAX model with holidays as exogenous variables
model_holds = SARIMAX(df_train_holidays['Vehicles'],
                order=(p, d, q),
                seasonal_order=(P, D, Q, s),
                exog=exog_train)
best_model_holds = model_holds.fit()

# Make predictions using the test exogenous variables
forecast_holds = best_model_holds.get_prediction(start=df_test.index[0], end=df_test.index[-1], exog=exog_test)
forecast_mean_holds = forecast_holds.predicted_mean
forecast_conf_int = forecast.conf_int()

In [ ]:
plt.figure(figsize=(10, 5))

# Plotting the training data
plt.plot(df_train.index, df_train, label='Train Data', color='blue')

# Plotting the actual test data
plt.plot(df_test.index, df_test, label='Test Data', color='green')

# Plotting the forecasted data
plt.plot(forecast_mean_holds.index, forecast_mean_holds, label='Forecast', color='red')

# Plotting the confidence intervals
plt.fill_between(forecast_conf_int.index,
                 forecast_conf_int.iloc[:, 0],
                 forecast_conf_int.iloc[:, 1], color='pink', alpha=0.3, label='Uncertainty Interval')

# Final plot adjustments
plt.title('SARIMA Model: Train, Test, and Forecast')
plt.xlabel('Date')
plt.ylabel('Vehicles')
plt.legend()
plt.show()

## Final Evaluation : Prophet - SARIMA - SARIMAX

In [ ]:
mae_sarimax = mean_absolute_error(df_test, forecast_mean_holds)
mape_sarimax = mean_absolute_percentage_error(df_test, forecast_mean_holds) * 100
rmse_sarimax = np.sqrt(mean_squared_error(df_test, forecast_mean_holds))

performance = pd.DataFrame({
    'Metrics': ['MAE', 'MAPE', 'RMSE'],
    'Prophet': [mae, f'{mape:.2f}%', rmse],
    'Prophet(holidays)': [mae_with_hols, f'{mape_with_hols:.2f}%', rmse_with_hols],
    'SARIMA': [mae_sarima, f'{mape_sarima:.2f}%', rmse_sarima],
    'SARIMAX(holidays)': [mae_sarimax, f'{mape_sarimax:.2f}%', rmse_sarimax]
})

print(performance.set_index('Metrics'))

The Prophet model with holidays provides the most accurate forecasts for this traffic dataset. Including holidays significantly improves the model's performance, especially in Prophet. While SARIMAX also benefits from including holidays, it does not outperform the Prophet models. This suggests that Prophet’s approach to handling holidays and trends is particularly well-suited to this data.